## AdaBoost

## Load data set

* https://archive.ics.uci.edu/ml/machine-learning-databases/adult/


* age: continuous.
* workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
* fnlwgt: continuous.
* education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
* education-num: continuous.
* marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
* occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
* relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
* race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
* sex: Female, Male.
* capital-gain: continuous.
* capital-loss: continuous.
* hours-per-week: continuous.
* native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

In [ ]:
pip install pandas

In [3]:
import pandas as pd

df = pd.read_csv("./data/adult.data",
                 names = ["age", "workclass", "fnlwgt", "education", "education-num", 
                          "marital-status", "occupation", "relationship",
                         "race", "sex", "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"], index_col=False)

In [4]:
df.head(10)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


In [5]:
len(df)

32561

## Simplify the data set

In [85]:
import numpy as np

def male_or_not(row):
    if row['sex'].lstrip() == 'Male':
        val = 'Yes'
    else:
        val = 'No'
    return val

def income_over_50(row):
    if row['income'].lstrip() == '>50K':
        val = 'Yes'
    else:
        val = 'No'
    return val

df['male'] = df.apply(male_or_not, axis=1)
df['>35 hours'] = np.where(df['hours-per-week']>35, 'Yes', 'No')
df['>40 years'] = np.where(df['age']>40, 'Yes', 'No')

# Target
df['>50k income'] = df.apply(income_over_50, axis=1)

df_simpl = df[['male', '>35 hours','>40 years','>50k income']]
df_simpl = df_simpl.head(10)
df_simpl

,male,>35 hours,>40 years,>50k income
0,Yes,Yes,No,No
1,Yes,No,Yes,No
2,Yes,Yes,No,No
3,Yes,Yes,Yes,No
4,No,Yes,No,No
5,No,Yes,No,No
6,No,No,Yes,No
7,Yes,Yes,Yes,Yes
8,No,Yes,No,Yes
9,Yes,Yes,Yes,Yes


## Calculate Gini Index

![title](img/images.svg)

In [96]:
def calc_weighted_gini_index(attribute):
    d_node = df_simpl[[attribute, '>50k income']]
    
    n = len(d_node)
    n_1 = len(d_node[d_node[attribute] == 'Yes'])
    n_2 = len(d_node[d_node[attribute] == 'No'])

    n_1_yes = len(d_node[(d_node[attribute] == 'Yes') & (d_node[">50k income"] == 'Yes')])
    n_1_no = len(d_node[(d_node[attribute] == 'Yes') & (d_node[">50k income"] == 'No')])
    n_2_yes = len(d_node[(d_node[attribute] == 'No') & (d_node[">50k income"] == 'Yes')])
    n_2_no = len(d_node[(d_node[attribute] == 'No') & (d_node[">50k income"] == 'No')])

    Gini_1 = 1-(n_1_yes/(n_1_yes + n_1_no)) ** 2-(n_1_no/(n_1_yes + n_1_no)) ** 2
    #print(f'Gini_1 = {Gini_1}')
    Gini_2 = 1-(n_2_yes/(n_2_yes + n_2_no)) ** 2-(n_2_no/(n_2_yes + n_2_no)) ** 2
    #print(f'Gini_2 = {Gini_2}')

    #weighted Gini impuraty for the selected attribute
    Gini_attribute = (n_1/n) * Gini_1 + (n_2/n) * Gini_2
    Gini_attribute = round(Gini_attribute, 3)
    print(f'Gini_{attribute} = {Gini_attribute}')
    
    return Gini_attribute

for attribute in df_simpl.columns[:-1]:
    calc_weighted_gini_index(attribute)

Gini_male = 0.417
Gini_>35 hours = 0.375
Gini_>40 years = 0.4


In [ ]:
for i in d.columns:
    print(i)

In [ ]:
!git add .
!git config git.mail "dmnkplzr@googlemail.com"
!git commit -m "update"
!git push